# ETL Pipeline Preparation
Follow the instructions below to help you create your ETL pipeline.
### 1. Import libraries and load datasets.
- Import Python libraries
- Load `messages.csv` into a dataframe and inspect the first few lines.
- Load `categories.csv` into a dataframe and inspect the first few lines.

In [1]:
# import libraries
import pandas as pd
from sqlalchemy import create_engine
import sys
import os
from nltk import word_tokenize
from nltk.corpus import stopwords
import re
from nltk.stem.wordnet import WordNetLemmatizer

In [2]:
# load messages dataset
messages = pd.read_csv('messages.csv')
messages.head()

,id,message,original,genre
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct


In [3]:
with open('messages.csv') as file:
    for i in range(10):
        line = file.readline()
        print('line: ', i,  line)
       

line:  0 id,message,original,genre

line:  1 2,Weather update - a cold front from Cuba that could pass over Haiti,Un front froid se retrouve sur Cuba ce matin. Il pourrait traverser Haiti demain. Des averses de pluie isolee sont encore prevues sur notre region ce soi,direct

line:  2 7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct

line:  3 8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ak timoun yo. Mesi se john jean depi Monben kwochi.",direct

line:  4 9,UN reports Leogane 80-90 destroyed. Only Hospital St. Croix functioning. Needs supplies desperately.,UN reports Leogane 80-90 destroyed. Only Hospital St. Croix functioning. Needs supplies desperately.,direct

line:  5 12,"says: west side of Haiti, rest of the country today and tonight",facade ouest d Haiti et le reste du pays aujourd hui et ce soir,direct

line:  6 14,Information about the National Palace-,Informtion au nivaux palais nationl,direct

line:  7 15,S

In [4]:
# load categories dataset
categories = pd.read_csv('categories.csv')
categories.head()

,id,categories
0,2,related-1;request-0;offer-0;aid_related-0;medi...
1,7,related-1;request-0;offer-0;aid_related-1;medi...
2,8,related-1;request-0;offer-0;aid_related-0;medi...
3,9,related-1;request-1;offer-0;aid_related-1;medi...
4,12,related-1;request-0;offer-0;aid_related-0;medi...


In [5]:
with open('categories.csv') as file:
    for i in range(10):
        line = file.readline()
        print('line: ', i,  line)

line:  0 id,categories

line:  1 2,related-1;request-0;offer-0;aid_related-0;medical_help-0;medical_products-0;search_and_rescue-0;security-0;military-0;child_alone-0;water-0;food-0;shelter-0;clothing-0;money-0;missing_people-0;refugees-0;death-0;other_aid-0;infrastructure_related-0;transport-0;buildings-0;electricity-0;tools-0;hospitals-0;shops-0;aid_centers-0;other_infrastructure-0;weather_related-0;floods-0;storm-0;fire-0;earthquake-0;cold-0;other_weather-0;direct_report-0

line:  2 7,related-1;request-0;offer-0;aid_related-1;medical_help-0;medical_products-0;search_and_rescue-0;security-0;military-0;child_alone-0;water-0;food-0;shelter-0;clothing-0;money-0;missing_people-0;refugees-0;death-0;other_aid-1;infrastructure_related-0;transport-0;buildings-0;electricity-0;tools-0;hospitals-0;shops-0;aid_centers-0;other_infrastructure-0;weather_related-1;floods-0;storm-1;fire-0;earthquake-0;cold-0;other_weather-0;direct_report-0

line:  3 8,related-1;request-0;offer-0;aid_related-0;medical

### 2. Merge datasets.
- Merge the messages and categories datasets using the common id
- Assign this combined dataset to `df`, which will be cleaned in the following steps

In [6]:
# merge datasets
df = pd.merge(messages,categories,how='outer',on=['id'])
df.head()

,id,message,original,genre,categories
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,related-1;request-0;offer-0;aid_related-0;medi...
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,related-1;request-0;offer-0;aid_related-1;medi...
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,related-1;request-0;offer-0;aid_related-0;medi...
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,related-1;request-1;offer-0;aid_related-1;medi...
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,related-1;request-0;offer-0;aid_related-0;medi...


In [7]:
# create a dataframe of the 36 individual category columns
categories = df['categories'].str.split(";",expand=True)
categories.head()

,0,1,2,3,4,5,6,7,8,9,...,26,27,28,29,30,31,32,33,34,35
0,related-1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
1,related-1,request-0,offer-0,aid_related-1,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-1,floods-0,storm-1,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
2,related-1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
3,related-1,request-1,offer-0,aid_related-1,medical_help-0,medical_products-1,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
4,related-1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0


In [8]:

row = categories.loc[0,:].unique()

category_colnames = [i[:-2] for i in categories.loc[0,:].unique()]
print(category_colnames)

['related', 'request', 'offer', 'aid_related', 'medical_help', 'medical_products', 'search_and_rescue', 'security', 'military', 'child_alone', 'water', 'food', 'shelter', 'clothing', 'money', 'missing_people', 'refugees', 'death', 'other_aid', 'infrastructure_related', 'transport', 'buildings', 'electricity', 'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure', 'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold', 'other_weather', 'direct_report']


In [9]:
# rename the columns of `categories`
categories.columns = category_colnames
categories.head().T

,0,1,2,3,4
related,related-1,related-1,related-1,related-1,related-1
request,request-0,request-0,request-0,request-1,request-0
offer,offer-0,offer-0,offer-0,offer-0,offer-0
aid_related,aid_related-0,aid_related-1,aid_related-0,aid_related-1,aid_related-0
medical_help,medical_help-0,medical_help-0,medical_help-0,medical_help-0,medical_help-0
medical_products,medical_products-0,medical_products-0,medical_products-0,medical_products-1,medical_products-0
search_and_rescue,search_and_rescue-0,search_and_rescue-0,search_and_rescue-0,search_and_rescue-0,search_and_rescue-0
security,security-0,security-0,security-0,security-0,security-0
military,military-0,military-0,military-0,military-0,military-0
child_alone,child_alone-0,child_alone-0,child_alone-0,child_alone-0,child_alone-0


In [10]:
categories.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,related-1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
1,related-1,request-0,offer-0,aid_related-1,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-1,floods-0,storm-1,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
2,related-1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
3,related-1,request-1,offer-0,aid_related-1,medical_help-0,medical_products-1,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
4,related-1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0


In [11]:
for column in categories:
    categories[column] = categories[column].apply(lambda x:x.split("-")[-1]).astype('int')
    
categories.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [12]:
df.head(1)

,id,message,original,genre,categories
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,related-1;request-0;offer-0;aid_related-0;medi...


In [13]:
# drop the original categories column from `df`
df.drop('categories',axis=1,inplace=True)


df.head()

,id,message,original,genre
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct


In [14]:
categories.head(1)

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
# concatenate the original dataframe with the new `categories` dataframe
df = pd.concat([df,categories],axis=1)
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [16]:
# check number of duplicates
(df[df.duplicated()].shape[0])

170

In [17]:
# drop duplicates
df= df[~df.duplicated()]

In [ ]:
engine = create_engine('sqlite:///FigureEight.db')
df.to_sql('FigureEight', engine, index=False)

In [29]:
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# ML Pipeline Preparation

In [40]:
# extract values from X and y
X =  df['message'].values
Y = df.iloc[:, 4:].values
category_names = list(df.columns[4:])

In [41]:
df['message'].values

array(['Weather update - a cold front from Cuba that could pass over Haiti',
       'Is the Hurricane over or is it not over',
       'Looking for someone but no name', ...,
       "Proshika, operating in Cox's Bazar municipality and 5 other unions, Ramu and Chokoria, assessment, 5 kg rice, 1,5 kg lentils to 700 families.",
       'Some 2,000 women protesting against the conduct of the elections were teargassed as they tried to converge on the local electoral commission offices in the southern oil city of Port Harcourt.',
       'A radical shift in thinking came about as a result of this meeting, recognizing that HIV/AIDS is at the core of the humanitarian crisis and identifying the crisis itself as a function of the HIV/AIDS pandemic.'],
      dtype=object)

In [42]:
def tokenize(text):
    """
    input:
        text: Message data for tokenization.
    output:
        clean_tokens: Result list after tokenization.
    """
    text = re.sub(r'[^A-Za-z0-9]'," ",text.lower())
    text = word_tokenize(text)
    words = [w.strip() for w in text if w not in stopwords.words("english")]
    lemmatizer = WordNetLemmatizer()
    clean_tokens = [WordNetLemmatizer().lemmatize(w) for w in words]
    
    return clean_tokens

In [73]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,f1_score,precision_recall_curve,accuracy_score
import numpy as np
from sklearn.multioutput import MultiOutputClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split, GridSearchCV

In [47]:
X_train, X_test, y_train, y_test = train_test_split(X, Y)
pipeline = Pipeline([('CountVect',CountVectorizer(tokenizer=tokenize)),
                     ('Tfidf',TfidfTransformer()),
                     ('MultiClass',MultiOutputClassifier(OneVsRestClassifier(LinearSVC())))])

pipeline.fit(X_train,y_train)
y_pred = pipeline.predict(X_test)

C:\Users\ofabiyi\Anaconda3\lib\site-packages\sklearn\multiclass.py:75: UserWarning: Label not 0 is present in all training examples.
  str(classes[c]))


In [87]:
for i in range(y_test.shape[1]):
    print('{} accuracy: {}'.format(category_colnames[i],accuracy_score(y_test[:,i],y_pred[:,i])))

related accuracy: 0.8199572779981691
request accuracy: 0.8985352456515106
offer accuracy: 0.9963381141287763
aid_related accuracy: 0.7644186756179432
medical_help accuracy: 0.9241684467500763
medical_products accuracy: 0.9534635337198657
search_and_rescue accuracy: 0.9766554775709491
security accuracy: 0.9821483063777846
military accuracy: 0.9710100701861458
child_alone accuracy: 1.0
water accuracy: 0.9584986267927983
food accuracy: 0.9450717119316447
shelter accuracy: 0.9452242905096124
clothing accuracy: 0.9900823924321025
money accuracy: 0.9766554775709491
missing_people accuracy: 0.9888617638083613
refugees accuracy: 0.9711626487641135
death accuracy: 0.9688739700945987
other_aid accuracy: 0.8602380225816295
infrastructure_related accuracy: 0.9333231614281355
transport accuracy: 0.9594140982606042
buildings accuracy: 0.9560573695453158
electricity accuracy: 0.9829111992676228
tools accuracy: 0.9935916997253585
hospitals accuracy: 0.9906927067439731
shops accuracy: 0.996490692706744

In [88]:
pipeline.get_params()

{'memory': None,
 'steps': [('CountVect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                   dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                   lowercase=True, max_df=1.0, max_features=None, min_df=1,
                   ngram_range=(1, 1), preprocessor=None, stop_words=None,
                   strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                   tokenizer=<function tokenize at 0x000001BA5DA35798>,
                   vocabulary=None)),
  ('Tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('MultiClass',
   MultiOutputClassifier(estimator=OneVsRestClassifier(estimator=LinearSVC(C=1.0,
                                                                           class_weight=None,
                                                                           dual=True,
                                                                           fit_intercept=Tru

In [90]:
parameters = {'CountVect__binary': [False,True],
              'Tfidf__norm': ['l1','l2'],
             'MultiClass__estimator__estimator__C': [1.0,3.0,5.0]
             }

cv = GridSearchCV(pipeline,param_grid=parameters)
cv.fit(X_train, y_train)
y_pred = cv.predict(X_test)

C:\Users\ofabiyi\Anaconda3\lib\site-packages\sklearn\multiclass.py:75: UserWarning: Label not 0 is present in all training examples.
  str(classes[c]))
C:\Users\ofabiyi\Anaconda3\lib\site-packages\sklearn\multiclass.py:75: UserWarning: Label not 0 is present in all training examples.
  str(classes[c]))
C:\Users\ofabiyi\Anaconda3\lib\site-packages\sklearn\multiclass.py:75: UserWarning: Label not 0 is present in all training examples.
  str(classes[c]))
C:\Users\ofabiyi\Anaconda3\lib\site-packages\sklearn\multiclass.py:75: UserWarning: Label not 0 is present in all training examples.
  str(classes[c]))
C:\Users\ofabiyi\Anaconda3\lib\site-packages\sklearn\multiclass.py:75: UserWarning: Label not 0 is present in all training examples.
  str(classes[c]))
C:\Users\ofabiyi\Anaconda3\lib\site-packages\sklearn\multiclass.py:75: UserWarning: Label not 0 is present in all training examples.
  str(classes[c]))
C:\Users\ofabiyi\Anaconda3\lib\site-packages\sklearn\multiclass.py:75: UserWarning: Labe

C:\Users\ofabiyi\Anaconda3\lib\site-packages\sklearn\multiclass.py:75: UserWarning: Label not 0 is present in all training examples.
  str(classes[c]))
C:\Users\ofabiyi\Anaconda3\lib\site-packages\sklearn\multiclass.py:75: UserWarning: Label not 0 is present in all training examples.
  str(classes[c]))
C:\Users\ofabiyi\Anaconda3\lib\site-packages\sklearn\multiclass.py:75: UserWarning: Label not 0 is present in all training examples.
  str(classes[c]))
C:\Users\ofabiyi\Anaconda3\lib\site-packages\sklearn\multiclass.py:75: UserWarning: Label not 0 is present in all training examples.
  str(classes[c]))
C:\Users\ofabiyi\Anaconda3\lib\site-packages\sklearn\multiclass.py:75: UserWarning: Label not 0 is present in all training examples.
  str(classes[c]))
C:\Users\ofabiyi\Anaconda3\lib\site-packages\sklearn\multiclass.py:75: UserWarning: Label not 0 is present in all training examples.
  str(classes[c]))
C:\Users\ofabiyi\Anaconda3\lib\site-packages\sklearn\multiclass.py:75: UserWarning: Labe

In [91]:
for i in range(y_test.shape[1]):
    print('{} accuracy: {}'.format(category_colnames[i],accuracy_score(y_test[:,i],y_pred[:,i])))

related accuracy: 0.8199572779981691
request accuracy: 0.8962465669819957
offer accuracy: 0.9963381141287763
aid_related accuracy: 0.7665547757094904
medical_help accuracy: 0.9259993896856882
medical_products accuracy: 0.9549893194995422
search_and_rescue accuracy: 0.9760451632590784
security accuracy: 0.9821483063777846
military accuracy: 0.9713152273420812
child_alone accuracy: 1.0
water accuracy: 0.9613976197741837
food accuracy: 0.9469026548672567
shelter accuracy: 0.9446139761977418
clothing accuracy: 0.9897772352761672
money accuracy: 0.9765028989929814
missing_people accuracy: 0.9887091852303936
refugees accuracy: 0.9708574916081782
death accuracy: 0.9682636557827281
other_aid accuracy: 0.8599328654256942
infrastructure_related accuracy: 0.9325602685382972
transport accuracy: 0.9591089411046689
buildings accuracy: 0.9574305767470247
electricity accuracy: 0.9835215135794935
tools accuracy: 0.9935916997253585
hospitals accuracy: 0.9905401281660055
shops accuracy: 0.996490692706744